In [ ]:
# 적합도 재검토

In [ ]:
# ---- Colab 업로드 위젯 (Colab 외 환경에선 무시) ----
try:
    from google.colab import files
    print("Upload '202501_clean2.xlsx'")
    up = files.upload()
    INPUT_FILENAME = "202501_clean2.xlsx" if "202501_clean2.xlsx" in up else next(
        (k for k in up.keys() if k.lower().endswith((".xlsx",".xls"))), None
    )
    if INPUT_FILENAME is None:
        raise FileNotFoundError("엑셀 파일(.xlsx/.xls)을 업로드하세요.")
except Exception:
    # 로컬/기타 환경
    INPUT_FILENAME = "202501_clean2.xlsx"

# ---- 기본 라이브러리 ----
import numpy as np, pandas as pd, warnings, math, json, re, os
from scipy import stats
from scipy.stats import (
    t, norm, logistic, gumbel_l, gumbel_r, weibull_min,
    beta as beta_dist, gamma as gamma_dist, lognorm, uniform,
    expon, triang, pareto, gaussian_kde, chi2
)
warnings.filterwarnings("ignore")

# ---- 사용자 설정 ----
METALS = ["Cr","Co","Ni","As","Cd","Sb","Pb"]     # 분석 순서 고정
RNG = np.random.default_rng(42)                    # 재현성: 단일 Generator 공유
BOOTSTRAP_B = 400                                  # 부트스트랩 반복(권장 400~1000)
MIN_EXPECTED = 5.0                                 # χ² 최소 기대빈도
OUT_XLSX = "Tx-적합도_FIXED.xlsx"

# ---- 유틸: 안전한 CDF 클리핑, KDE 모드, 정렬 ----
def _clip01(p, eps=1e-12):
    return np.minimum(1 - eps, np.maximum(eps, p))

def kde_mode(x):
    # KDE 기반 모드 추정 (삼각형/BetaPERT 등에 사용)
    kde = gaussian_kde(x)
    grid = np.linspace(np.min(x), np.max(x), 512)
    return grid[np.argmax(kde(grid))]

# ---- A-D 통계량 (원형, EDF) ----
def ad_stat(x, cdf):
    # x: 1D 실수 배열 (정렬·결측 제거 필요)
    x = np.asarray(x, float)
    x = x[np.isfinite(x)]
    x = np.sort(x)
    n = x.size
    if n < 5: return np.nan
    Fi = _clip01(cdf(x))
    i = np.arange(1, n+1)
    A2 = -n - np.mean((2*i - 1) * (np.log(Fi) + np.log(1 - Fi[::-1])))
    return float(A2)

# ---- K-S 통계량(D)만 SciPy로, p-값은 부트스트랩 ----
def ks_stat_only(x, cdf):
    try:
        D = stats.kstest(x, lambda z: cdf(z)).statistic
        return float(D)
    except Exception:
        return np.nan

# ---- χ²: 분위수 bin(동일 기대빈도), df = bins-1-k_eff ----
def chi2_stat_p(x, dist_name, params, ppf, n_params_effective):
    x = np.asarray(x, float)
    x = x[np.isfinite(x)]
    n = x.size
    if n < 20:
        return np.nan, np.nan
    # bin 수: 최소기대빈도 ≥ MIN_EXPECTED를 보장하기 위해 최대 bin = floor(n / MIN_EXPECTED)
    max_bins = int(max(5, min(50, n // MIN_EXPECTED)))
    if max_bins < 3:
        return np.nan, np.nan
    eps = 1e-8
    # 분위수 경계 (동일 기대빈도)
    qs = np.linspace(eps, 1 - eps, max_bins + 1)
    edges = ppf(qs, params)
    edges = np.unique(edges)  # 중복 제거
    if edges.size < 3:
        return np.nan, np.nan
    obs, _ = np.histogram(x, bins=edges)
    exp = np.full_like(obs, fill_value=n / obs.size, dtype=float)  # 동일 기대빈도

    # 자유도: bins - 1 - k_eff
    k = n_params_effective(dist_name, params)
    df = obs.size - 1 - k
    if df <= 0:
        return np.nan, np.nan

    # 기대빈도 보정(혹여 0 방지)
    exp = np.maximum(exp, 1e-12)
    chi = float(np.sum((obs - exp) ** 2 / exp))
    p = float(1.0 - chi2.cdf(chi, df))
    return chi, p

# ---- 분포별 래퍼: fit / cdf / ppf / rvs / k_eff ----
# * SciPy .fit은 기본적으로 loc/scale 자유(3-모수 허용) → "shift" 허용
# * Beta/Uniform/Triangular/BetaPERT는 [a,b] 경계 추정 포함 → 자유도에 반영
def fit_student_t(x):
    return t.fit(x)  # (df, loc, scale)

def cdf_student_t(z, p):
    df_, loc, sc = p
    return t.cdf(z, df_, loc=loc, scale=sc)

def ppf_student_t(q, p):
    df_, loc, sc = p
    return t.ppf(q, df_, loc=loc, scale=sc)

def rvs_student_t(n, p):
    df_, loc, sc = p
    return t.rvs(df_, loc=loc, scale=sc, size=n, random_state=RNG)

def fit_norm(x):    return norm.fit(x)
def cdf_norm(z,p):  return norm.cdf(z, *p)
def ppf_norm(q,p):  return norm.ppf(q, *p)
def rvs_norm(n,p):  return norm.rvs(*p, size=n, random_state=RNG)

def fit_logistic(x):   return logistic.fit(x)
def cdf_logistic(z,p): return logistic.cdf(z, *p)
def ppf_logistic(q,p): return logistic.ppf(q, *p)
def rvs_logistic(n,p): return logistic.rvs(*p, size=n, random_state=RNG)

def fit_gumbel_l(x):   return gumbel_l.fit(x)     # (loc, scale)
def cdf_gumbel_l(z,p): return gumbel_l.cdf(z, *p)
def ppf_gumbel_l(q,p): return gumbel_l.ppf(q, *p)
def rvs_gumbel_l(n,p): return gumbel_l.rvs(*p, size=n, random_state=RNG)

def fit_gumbel_r(x):   return gumbel_r.fit(x)
def cdf_gumbel_r(z,p): return gumbel_r.cdf(z, *p)
def ppf_gumbel_r(q,p): return gumbel_r.ppf(q, *p)
def rvs_gumbel_r(n,p): return gumbel_r.rvs(*p, size=n, random_state=RNG)

def fit_weibull(x):    return weibull_min.fit(x)  # (c, loc, scale)
def cdf_weibull(z,p):  return weibull_min.cdf(z, *p)
def ppf_weibull(q,p):  return weibull_min.ppf(q, *p)
def rvs_weibull(n,p):  return weibull_min.rvs(*p, size=n, random_state=RNG)

def fit_beta_scaled(x):
    a = float(np.min(x)); b = float(np.max(x))
    if not (np.isfinite(a) and np.isfinite(b) and b > a):
        raise RuntimeError("Invalid support for Beta.")
    z = (x - a) / (b - a)
    z = np.clip(z, 1e-9, 1 - 1e-9)
    alpha, beta_, loc, scale = beta_dist.fit(z, floc=0, fscale=1)  # α,β만 추정
    return (alpha, beta_, a, b)

def cdf_beta_scaled(z, p):
    alpha, beta_, a, b = p
    return beta_dist.cdf((z - a) / (b - a), alpha, beta_)

def ppf_beta_scaled(q, p):
    alpha, beta_, a, b = p
    return a + (b - a) * beta_dist.ppf(q, alpha, beta_)

def rvs_beta_scaled(n, p):
    alpha, beta_, a, b = p
    r = beta_dist.rvs(alpha, beta_, size=n, random_state=RNG)
    return a + (b - a) * r

def fit_gamma(x):     return gamma_dist.fit(x)    # (a, loc, scale)
def cdf_gamma(z,p):   return gamma_dist.cdf(z, *p)
def ppf_gamma(q,p):   return gamma_dist.ppf(q, *p)
def rvs_gamma(n,p):   return gamma_dist.rvs(*p, size=n, random_state=RNG)

def fit_lognorm(x):   return lognorm.fit(x)       # (s, loc, scale)
def cdf_lognorm(z,p): return lognorm.cdf(z, *p)
def ppf_lognorm(q,p): return lognorm.ppf(q, *p)
def rvs_lognorm(n,p): return lognorm.rvs(*p, size=n, random_state=RNG)

def fit_uniform_scaled(x):
    a = float(np.min(x)); b = float(np.max(x))
    if not (np.isfinite(a) and np.isfinite(b) and b > a):
        raise RuntimeError("Invalid support for Uniform.")
    return (a, b - a)  # (loc, scale)

def cdf_uniform_scaled(z,p): return uniform.cdf(z, *p)
def ppf_uniform_scaled(q,p): return uniform.ppf(q, *p)
def rvs_uniform_scaled(n,p): return uniform.rvs(*p, size=n, random_state=RNG)

def fit_expon(x):     return expon.fit(x)         # (loc, scale)
def cdf_expon(z,p):   return expon.cdf(z, *p)
def ppf_expon(q,p):   return expon.ppf(q, *p)
def rvs_expon(n,p):   return expon.rvs(*p, size=n, random_state=RNG)

def fit_triang(x):
    a = float(np.min(x)); b = float(np.max(x))
    if not (np.isfinite(a) and np.isfinite(b) and b > a):
        raise RuntimeError("Invalid support for Triangular.")
    m = float(np.clip(kde_mode(x), a + 1e-9, b - 1e-9))
    c = (m - a) / (b - a)
    return (c, a, b - a)  # (c, loc=a, scale=b-a)

def cdf_triang(z,p):  return triang.cdf(z, *p)
def ppf_triang(q,p):  return triang.ppf(q, *p)
def rvs_triang(n,p):  return triang.rvs(*p, size=n, random_state=RNG)

def fit_betapert(x, lam=4.0):
    a = float(np.min(x)); b = float(np.max(x))
    if not (np.isfinite(a) and np.isfinite(b) and b > a):
        raise RuntimeError("Invalid support for BetaPERT.")
    m = float(np.clip(kde_mode(x), a + 1e-9, b - 1e-9))
    alpha = 1 + lam * (m - a) / (b - a)
    beta_ = 1 + lam * (b - m) / (b - a)
    return (alpha, beta_, a, b, m, lam)  # 파라미터 기록

def cdf_betapert(z,p):
    alpha, beta_, a, b, m, lam = p
    return beta_dist.cdf((z - a) / (b - a), alpha, beta_)

def ppf_betapert(q,p):
    alpha, beta_, a, b, m, lam = p
    return a + (b - a) * beta_dist.ppf(q, alpha, beta_)

def rvs_betapert(n,p):
    alpha, beta_, a, b, m, lam = p
    r = beta_dist.rvs(alpha, beta_, size=n, random_state=RNG)
    return a + (b - a) * r

def fit_pareto(x):    return pareto.fit(x)        # (b, loc, scale)
def cdf_pareto(z,p):  return pareto.cdf(z, *p)
def ppf_pareto(q,p):  return pareto.ppf(q, *p)
def rvs_pareto(n,p):  return pareto.rvs(*p, size=n, random_state=RNG)

# ---- 분포 레지스트리 ----
DISTS = [
    ("Student's t",            fit_student_t,   cdf_student_t,   ppf_student_t,   rvs_student_t),
    ("Normal",                 fit_norm,        cdf_norm,        ppf_norm,        rvs_norm),
    ("Logistic",               fit_logistic,    cdf_logistic,    ppf_logistic,    rvs_logistic),
    ("Minimum Extreme Value",  fit_gumbel_l,    cdf_gumbel_l,    ppf_gumbel_l,    rvs_gumbel_l),
    ("Maximum Extreme Value",  fit_gumbel_r,    cdf_gumbel_r,    ppf_gumbel_r,    rvs_gumbel_r),
    ("Weibull",                fit_weibull,     cdf_weibull,     ppf_weibull,     rvs_weibull),
    ("Beta",                   fit_beta_scaled, cdf_beta_scaled, ppf_beta_scaled, rvs_beta_scaled),
    ("Gamma",                  fit_gamma,       cdf_gamma,       ppf_gamma,       rvs_gamma),
    ("Lognormal",              fit_lognorm,     cdf_lognorm,     ppf_lognorm,     rvs_lognorm),
    ("Uniform",                fit_uniform_scaled, cdf_uniform_scaled, ppf_uniform_scaled, rvs_uniform_scaled),
    ("Exponential",            fit_expon,       cdf_expon,       ppf_expon,       rvs_expon),
    ("Triangular",             fit_triang,      cdf_triang,      ppf_triang,      rvs_triang),
    ("BetaPERT",               fit_betapert,    cdf_betapert,    ppf_betapert,    rvs_betapert),
    ("Pareto",                 fit_pareto,      cdf_pareto,      ppf_pareto,      rvs_pareto),
]

# ---- 자유도(k_eff): 추정 파라미터 개수(경계 포함) ----
def k_eff(dist_name, params):
    dn = dist_name.lower()
    if dn == "student's t":        return 3   # df, loc, scale
    if dn in ("normal","logistic"):return 2   # loc, scale
    if "extreme" in dn:            return 2   # gumbel_l/r: loc, scale
    if dn in ("weibull","lognormal","gamma","pareto"): return 3  # shape, loc, scale
    if dn == "exponential":        return 2   # loc, scale
    if dn == "uniform":            return 2   # [a,b] -> loc, scale
    if dn == "triangular":         return 3   # a,m,b -> (c,loc,scale) 3
    if dn == "betapert":           return 3   # a,m,b (lam 고정)
    if dn == "beta":
        # scaled Beta: alpha,beta + a,b 경계
        return 4
    # fallback
    return 3

# ---- 부트스트랩 p-값 (A-D, K-S) : 매 반복 재적합 ----
def p_bootstrap_stat(x, fit, cdf, rvs, stat_func, B=BOOTSTRAP_B):
    x = np.asarray(x, float)
    x = x[np.isfinite(x)]
    n = x.size
    if n < 5:
        return np.nan, np.nan  # (stat, p)
    # 관측치 적합
    params = fit(x)
    obs = stat_func(x, lambda z: cdf(z, params))
    if not np.isfinite(obs):
        return obs, np.nan
    ge = 0
    for _ in range(B):
        xb = rvs(n, params)
        try:
            pb = fit(xb)
            sb = stat_func(xb, lambda z: cdf(z, pb))
            if np.isfinite(sb) and sb >= obs:
                ge += 1
        except Exception:
            pass
    # small-sample correction
    p = (ge + 1) / (B + 1)
    return float(obs), float(p)

# ---- 데이터 로드 & 금속 컬럼 매핑 ----
raw = pd.read_excel(INPUT_FILENAME)
colmap = {}
for m in METALS:
    exact = [c for c in raw.columns if c == f"{m}(ng/m3)"]
    cand  = exact or [c for c in raw.columns if m in c]
    if cand: colmap[m] = cand[0]

# ---- 메인 루프 ----
detail_by_metal = []
summary_rows = []

for m in METALS:
    col = colmap.get(m)
    if not col:
        continue
    x0 = pd.to_numeric(raw[col], errors="coerce").values
    x  = x0[np.isfinite(x0)]
    if x.size < 20:
        continue

    rows = []
    for name, fit_f, cdf_f, ppf_f, rvs_f in DISTS:
        try:
            # 모수 적합(관측치)
            params = fit_f(x)

            # A-D (stat & bootstrap p)
            A2, pA = p_bootstrap_stat(x, fit_f, cdf_f, rvs_f, ad_stat, B=BOOTSTRAP_B)

            # K-S (stat & bootstrap p) - 통계량은 SciPy kstest에서 계산
            def _ks_stat(x_in, cdf_):
                return ks_stat_only(x_in, cdf_)
            D, pK = p_bootstrap_stat(x, fit_f, cdf_f, rvs_f, _ks_stat, B=BOOTSTRAP_B)

            # Chi-square
            chi, pC = chi2_stat_p(
                x, name, params,
                ppf=lambda q, p: ppf_f(q, p),
                n_params_effective=k_eff
            )

            # 파라미터 문자열(요약)
            def pstr():
                try:
                    if name == "Lognormal":
                        s, loc, sc = params; return f"s={s:.5g}, loc={loc:.5g}, scale={sc:.5g}"
                    if name == "Weibull":
                        c, loc, sc = params; return f"c={c:.5g}, loc={loc:.5g}, scale={sc:.5g}"
                    if name == "Gamma":
                        a, loc, sc = params; return f"a={a:.5g}, loc={loc:.5g}, scale={sc:.5g}"
                    if name == "Exponential":
                        loc, sc = params;   return f"loc={loc:.5g}, scale={sc:.5g}"
                    if name in ("Minimum Extreme Value","Maximum Extreme Value","Normal","Logistic"):
                        loc, sc = params;   return f"loc={loc:.5g}, scale={sc:.5g}"
                    if name == "Student's t":
                        df_, loc, sc = params; return f"df={df_:.5g}, loc={loc:.5g}, scale={sc:.5g}"
                    if name == "Pareto":
                        b, loc, sc = params; return f"b={b:.5g}, loc={loc:.5g}, scale={sc:.5g}"
                    if name == "Uniform":
                        loc, sc = params;   return f"a={loc:.5g}, b={(loc+sc):.5g}"
                    if name == "Triangular":
                        c, loc, sc = params; a=loc; b=loc+sc; m=a+c*(b-a); return f"a={a:.5g}, m={m:.5g}, b={b:.5g}"
                    if name == "BetaPERT":
                        alpha, beta_, a, b, m_, lam = params; return f"a={a:.5g}, m={m_:.5g}, b={b:.5g}, alpha={alpha:.5g}, beta={beta_:.5g}, lam={lam:.5g}"
                    if name == "Beta":
                        alpha, beta_, a, b = params; return f"a={a:.5g}, b={b:.5g}, alpha={alpha:.5g}, beta={beta_:.5g}"
                except Exception:
                    return json.dumps(params)
                return json.dumps(params)

            rows.append({
                "Metal": m,
                "Distribution": name,
                "Anderson-Darling": A2,
                "A-D P-value": pA,
                "K-S": D,
                "K-S P-value": pK,
                "Chi-square": chi,
                "Chi-square P-value": pC,
                "Params": pstr()
            })
        except Exception as e:
            rows.append({
                "Metal": m,
                "Distribution": name,
                "Anderson-Darling": np.nan,
                "A-D P-value": np.nan,
                "K-S": np.nan,
                "K-S P-value": np.nan,
                "Chi-square": np.nan,
                "Chi-square P-value": np.nan,
                "Params": f"ERROR: {str(e)}"
            })

    metal_df = pd.DataFrame(rows)
    # 가독성: A-D 오름차순(작을수록 적합), 동일 시 K-S 오름차순
    metal_df = metal_df.sort_values(["Anderson-Darling","K-S"], na_position="last").reset_index(drop=True)
    detail_by_metal.append((m, metal_df))
    summary_rows.append(metal_df.assign(Metal=m))

summary = pd.concat([df for _, df in detail_by_metal], ignore_index=True)

# ---- 엑셀 저장 ----
with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as writer:
    # 금속별 시트
    for m, dfm in detail_by_metal:
        # p-값 문자열 컬럼(가독)
        out = dfm.copy()
        for col in ["A-D P-value","K-S P-value","Chi-square P-value"]:
            out[col + " (str)"] = out[col].apply(lambda v: "" if pd.isna(v) else f"{v:.4g}")
        out.to_excel(writer, index=False, sheet_name=m)
    # Summary
    s = summary.copy()
    for col in ["A-D P-value","K-S P-value","Chi-square P-value"]:
        s[col + " (str)"] = s[col].apply(lambda v: "" if pd.isna(v) else f"{v:.4g}")
    s.to_excel(writer, index=False, sheet_name="Summary")

with pd.ExcelWriter(OUT_XLSX, engine="openpyxl") as writer:
    # 금속별 시트
    for m, dfm in detail_by_metal:
        out = dfm.copy()
        for col in ["A-D P-value","K-S P-value","Chi-square P-value"]:
            out[col + " (str)"] = out[col].apply(lambda v: "" if pd.isna(v) else f"{v:.4g}")
        out.to_excel(writer, index=False, sheet_name=m)
    # Summary 시트
    s = summary.copy()
    for col in ["A-D P-value","K-S P-value","Chi-square P-value"]:
        s[col + " (str)"] = s[col].apply(lambda v: "" if pd.isna(v) else f"{v:.4g}")
    s.to_excel(writer, index=False, sheet_name="Summary")

print("Saved:", OUT_XLSX)

# Colab이면 자동 다운로드
try:
    from google.colab import files
    files.download(OUT_XLSX)
except Exception:
    pass

Upload '202501_clean2.xlsx'


Saving 202501_clean2.xlsx to 202501_clean2 (8).xlsx
